In [1]:
using CSV
using LinearAlgebra,Statistics,Random,DelimitedFiles, DataFrames

In [2]:
function writeMatBin(inFileName,outFileName;header=true,dlm=' ',missingStr="NA")
    inFile   = open(inFileName, "r")
    n    = countlines(inFile)
    close(inFile)
    inFile   = open(inFileName, "r")
    line = readline(inFile)
    res  = split(line,dlm)
    p    = size(res,1)
    close(inFile)
 
    if header==true
        M = Array{Union{Float64,Missings.Missing}}(undef,p-1,n-1)
        j = 0
    else
        M = Array{Union{Float64,Missings.Missing}}(undef,p-1,n)
        ids = Array{String}(undef,n)            
        j = 1
    end
    outID = open(outFileName*".ids.txt","w")     
    for line in eachline(inFileName)
        if j > 0 # skip header
            if j%100 == 0 
                println(j)
            end
            res  = split(line,dlm)
            println(outID,res[1])   
            for i in 2:p 
                x = res[i]
                M[i-1,j] = x!=missingStr ? parse(Float64,x) : Missings.missing
            end
        end
        j += 1
    end

    replaceMissing!(M)
    M = convert(Array{Float64}, M)
    if header==true
        write(outFileName*".gen.bin",n-1,p-1,M')
    else
        write(outFileName*".gen.bin",n-1,p-1,M')
    end
    close(outID)            
end

writeMatBin (generic function with 1 method)

In [3]:
function readMatBin(fileName)
    genStr = open(fileName)
    n = read(genStr,Int64)
    p = read(genStr,Int64)
    M = zeros(n,p)
    for j in 1:p
        for i in 1:n
            M[i,j] = read(genStr,Float64)
        end
    end
    close(genStr)
    return M
end

readMatBin (generic function with 1 method)

In [4]:
writeMatBin("ALGP2_Cycle_1_7_Geno_Complete_JWAS.txt","ALGP2_Cycle_1_7_Geno_Complete_JWAS.bin",missingStr=".")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


UndefVarError: UndefVarError: replaceMissing! not defined

In [ ]:
M = readMatBin("Geno_JWAS_impute.bin.gen.bin")

In [3]:
# impute missing snps
function replaceMissing!(M)
    for j=1:size(M,2)
        jMean = mean(skipmissing(M[:,j]))
        M[ismissing.(M[:,j]),j] .= jMean
    end
end

replaceMissing! (generic function with 1 method)